# Get samba related scripts
> Samba related tools

In [ ]:
#| default_exp data_processing.smb_tools

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2


In [ ]:
#| export
import sys
from pathlib import Path


In [ ]:
#| export
CV_TOOLS = Path(r'/home/ai_sintercra/homes/hasan/projects/git_data/cv_tools')
sys.path.append(str(CV_TOOLS))


In [ ]:
#| export
custom_lib_path = Path(r'/home/ai_warstein/homes/goni/custom_libs')
sys.path.append(str(custom_lib_path))


In [ ]:
#| export
from cv_tools.imports import *
from dotenv import load_dotenv
from smb.SMBConnection import SMBConnection
from typing import List, Dict, Any

In [ ]:
#| export
load_dotenv(dotenv_path=f'/home/ai_sintercra/homes/hasan/projects/git_data/cv_tools/cv_tools/.env')

False

In [ ]:
#| export
def get_user_name_password(dotenv_path: str=None):
    load_dotenv(dotenv_path=dotenv_path)
    USERNAME = os.getenv('ServiceAccUsername')
    PASSWORD = os.getenv('ServiceAccPassword')
    return USERNAME, PASSWORD


In [ ]:
#| export
def get_smb_filename(
    SERVER: str='MUCSDN57.eu.infineon.com',
    SHARE: str='earchive8$',
    SMB_PATH: str=r"Business_Transfer\WAR",
    filter_ext: str=None,
    target_path: str=None,
    USERNAME: str=None,
    PASSWORD: str=None,
    ) -> List[str]:


    conn = SMBConnection(
        username=USERNAME, 
        password=PASSWORD, 
        my_name=SHARE, 
        remote_name=SERVER, 
        use_ntlm_v2=True)
    conn.connect(SERVER, 139)

    try:
        # Corrected the path to match the expected SMB path format
        file_list = conn.listPath(SHARE, SMB_PATH)
        file_list = list(filter(lambda x: not(x.filename.startswith(".")), file_list))
        if filter_ext:
            file_list = list(filter(lambda x: x.filename.endswith(filter_ext), file_list))
        file_list_names = []
        for fn in file_list:
            ## Constructing the SMB path correctly for each file
            s_path = SMB_PATH + "\\" + fn.filename
            t_path = Path(target_path, fn.filename)
            f_sz = fn.file_size
            t_path.parent.mkdir(parents=True, exist_ok=True)
            if Path(t_path).is_file():
                if Path(t_path).stat().st_size != f_sz:
                    file_list_names.append(f"{s_path}|{t_path}")
            else:
                file_list_names.append(f"{s_path}|{t_path}")
        print(len(file_list_names), len(file_list))
        return file_list_names, file_list
    
    finally:
        conn.close()

In [ ]:
#| export
def download_single_file(
    smb_filename: str, 
    SHARE: str,
    SERVER: str,
    USERNAME: str,
    PASSWORD: str,
    delete_source: bool = False) -> Dict[str, Any]:
    """
    Downloads a single file from SMB to a local directory and optionally deletes the source file.

    Args:
    - smb_filename (str): The SMB path of the file to download, followed by a "|" and the target local path.
    - SHARE (str): SMB share name
    - SERVER (str): SMB server name
    - USERNAME (str): SMB username
    - PASSWORD (str): SMB password 
    - delete_source (bool): Whether to delete source file after download (default: False)

    Returns:
    - Dict containing file metadata (name, size)

    Example:
    >>> download_single_file("smb://path/to/file.txt|local/path/to/file.txt")
    """
    source, target_name = smb_filename.split("|")
    target_path = Path(target_name)
    target_path.parent.mkdir(parents=True, exist_ok=True)
    
    conn = SMBConnection(
        USERNAME, 
        PASSWORD, 
        remote_name=SERVER, 
        my_name=SHARE,
        use_ntlm_v2=True)
    conn.connect(SERVER, 139)

    file_info = {}
    try:
        # Get file info before downloading
        file_obj = conn.getAttributes(SHARE, source)
        file_info = {
            'filename': source.split('\\')[-1],
            'size': file_obj.file_size,
            'create_time': file_obj.create_time,
            'last_modified': file_obj.last_write_time
        }
        
        # Download file
        with open(target_name, 'wb') as fp:
            conn.retrieveFile(SHARE, source, fp)

        print(f"File downloaded successfully: {target_name}")
        print(f"{'-'*100}")
        print(f"{'-'*100}")  # Normal color
        # Delete source if requested
        if delete_source:
            print('\033[91m' + "Deleting source file: " + source + '\033[0m')  # Red color
            conn.deleteFiles(SHARE, source)
            
        return file_info
            
    finally:
        conn.close()

In [ ]:
#| export
CURRETNT_NB='/home/ai_sintercra/homes/hasan/projects/git_data/cv_tools/nbs'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export('09_data_processing.smb_tools.ipynb')